# Initialization

In [1]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget --directory-prefix=/content -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
# !wget --directory-prefix=/content -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
# !tar xf spark-2.4.5-bin-hadoop2.7.tgz
# !pip install -q findspark

In [2]:
# import os
# import findspark

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# findspark.init("/content/spark-2.4.5-bin-hadoop2.7")

# BASE_URL = "./drive/My Drive/Colab Notebooks"

# Import Packages

In [3]:
# Pyspark Packages
from pyspark.sql import SparkSession, Window, Row
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import functions as F, DataFrameReader
from pyspark.sql.types import (
    StringType,
    ArrayType,
    IntegerType,
    FloatType,
    StructType,
    StructField,
    BooleanType,
    MapType,
    DateType
)

In [4]:
def config():

    conf = SparkConf()
    # COMMON CONFIGURATION
    conf.set("spark.sql.caseSensitive", "true")
    conf.set("spark.debug.maxToStringFields", 200)
    conf.set("spark.sql.files.ignoreCorruptFiles", "true")
    conf.set('spark.sql.session.timeZone', 'Asia/Jakarta')

    return conf

In [5]:
conf = config()
spark = SparkSession \
    .builder \
    .config(conf=conf) \
    .appName("EML Batch 9") \
    .getOrCreate()
sc = spark.sparkContext

# Read Data

In [6]:
df = spark.read.csv(f"/Users/muhammadramadiansyah/Downloads/retail.csv",inferSchema=True, header =True)
df.show()

+------+--------------+----------+---------+--------------+-----------+-----------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date|Ship Date|     Ship Mode|Customer ID|    Segment|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+---------+--------------+-----------+-----------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156|   11/8/16| 11/11/16|  Second Class|   CG-12520|   Consumer|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset Col...|  261.96|       2|       0| 41.9136|
|     2|CA-2016-152156|   11/8/16| 11/11/16|  Second Class|   CG-12520|   Consumer|FUR-CH-10000454|      Furniture|      Chairs|Hon Deluxe Fabric...|  731.94|       3|       0| 219.582|
|     3|CA-2016-138688|   6/12/16|  6/16/16|  Second Class|   DV-13045

In [7]:
df_filtered = df \
      .dropna(subset=["Product ID"]) \
      .groupBy("Order ID") \
      .agg(F.collect_list("Product Name").alias("Product List")) \
      .withColumn("Product List", F.udf(lambda x: list(set(x)), ArrayType(StringType()))(F.col("Product List")))
df_filtered.show()

+--------------+--------------------+
|      Order ID|        Product List|
+--------------+--------------------+
|CA-2014-114335|[Deflect-o DuraMa...|
|CA-2014-120670|[Razer Kraken 7.1...|
|CA-2014-141838|[Griffin GC17055 ...|
|CA-2015-115798|[Innergie mMini C...|
|CA-2015-116750|[Wilson Jones Tur...|
|CA-2015-128083|[Eldon Expression...|
|CA-2015-144253|[Electrix 20W Hal...|
|CA-2015-161830|[BOSTON Model 180...|
|CA-2015-166604|[Wausau Papers As...|
|CA-2016-124016|[Fellowes Black P...|
|CA-2016-134936|[Wilson Jones Cli...|
|CA-2016-135776|[Newell 317, Stap...|
|CA-2016-145730|[Canon PC170 Desk...|
|CA-2016-149223|[Fellowes Basic H...|
|CA-2016-149783|[Prismacolor Colo...|
|CA-2017-103520|        [Xerox 1899]|
|CA-2017-113474|[Multimedia Mailers]|
|CA-2017-122994|[Bush Andora Book...|
|CA-2017-126221|[Eureka The Boss ...|
|CA-2017-132521|[Avery Printable ...|
+--------------+--------------------+
only showing top 20 rows



# Modeling

In [8]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="Product List", minSupport=0.0001, minConfidence=0.0001)
model = fpGrowth.fit(df_filtered)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df_filtered).show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Logitech Trackma...|   9|
|[Logitech Trackma...|   1|
|[Logitech Trackma...|   1|
|[Logitech Trackma...|   1|
|[Logitech Trackma...|   1|
|[Logitech Trackma...|   1|
|[Logitech Trackma...|   1|
|[Logitech Trackma...|   1|
|[Logitech Trackma...|   1|
|[Kensington 4 Out...|   7|
|[Kensington 4 Out...|   1|
|[Kensington 4 Out...|   1|
|[Kensington 4 Out...|   1|
|[Kensington 4 Out...|   1|
|[Kensington 4 Out...|   1|
|[Kensington 4 Out...|   1|
|[Kensington 4 Out...|   1|
|[Kensington 4 Out...|   1|
|         [Xerox 210]|   6|
|[Xerox 210, Maste...|   1|
+--------------------+----+
only showing top 20 rows

+--------------------+--------------------+----------+------------------+
|          antecedent|          consequent|confidence|              lift|
+--------------------+--------------------+----------+------------------+
|[StarTech.com 10/...|[DAX Black Cherry...|       1.0|           626.125|
|[Star

# Filtered DataFrame

In [9]:
df_filtered = df \
      .dropna(subset=["Product ID"]) \
      .groupBy("Order ID") \
      .agg(
          F.collect_list("Sub-Category").alias("Sub-Category List"),
          F.collect_list("Product Name").alias("Product Name List")
        ) \
      .withColumn("Sub-Category List", F.udf(lambda x: list(set(x)), ArrayType(StringType()))(F.col("Sub-Category List")))
df_filtered.show()

+--------------+--------------------+--------------------+
|      Order ID|   Sub-Category List|   Product Name List|
+--------------+--------------------+--------------------+
|CA-2014-114335|       [Furnishings]|[Deflect-o DuraMa...|
|CA-2014-120670|       [Accessories]|[Razer Kraken 7.1...|
|CA-2014-141838|[Phones, Binders,...|[Newell 308, Wils...|
|CA-2015-115798|[Bookcases, Phone...|[BlackBerry Q10, ...|
|CA-2015-116750|[Binders, Furnish...|[Stackable Trays,...|
|CA-2015-128083|[Envelopes, Furni...|[Convenience Pack...|
|CA-2015-144253|       [Furnishings]|[Electrix 20W Hal...|
|CA-2015-161830|      [Binders, Art]|[BOSTON Model 180...|
|CA-2015-166604|         [Envelopes]|[Wausau Papers As...|
|CA-2016-124016|[Paper, Binders, ...|[Xerox 1970, Eldo...|
|CA-2016-134936|   [Binders, Tables]|[SAFCO PlanMaster...|
|CA-2016-135776|[Envelopes, Paper...|[Newell 317, Comp...|
|CA-2016-145730|[Machines, Envelo...|["Office Impressi...|
|CA-2016-149223|        [Appliances]|[Fellowes Basic H..

In [10]:
from pyspark.ml.fpm import FPGrowth

# df = spark.createDataFrame([
#     (0, ["1", "2", "a"]),
#     (1, ["1", "2", "3", "a"]),
#     (2, ["1", "2"])
# ], ["id", "items"])

fpGrowth = FPGrowth(itemsCol="Sub-Category List", minSupport=0.001, minConfidence=0.4)
model = fpGrowth.fit(df_filtered)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
df_output = model.transform(df_filtered)
df_output.show(20, False, vertical=True)

+--------------------+----+
|               items|freq|
+--------------------+----+
|           [Binders]|1316|
|             [Paper]|1191|
|    [Paper, Binders]| 275|
|       [Furnishings]| 877|
|[Furnishings, Paper]| 177|
|[Furnishings, Pap...|  51|
|[Furnishings, Bin...| 198|
|            [Phones]| 814|
|[Phones, Furnishi...| 148|
|[Phones, Furnishi...|  42|
|[Phones, Furnishi...|  11|
|[Phones, Furnishi...|  40|
|     [Phones, Paper]| 175|
|[Phones, Paper, B...|  54|
|   [Phones, Binders]| 199|
|           [Storage]| 777|
|[Storage, Furnish...| 138|
|[Storage, Furnish...|  46|
|[Storage, Furnish...|  13|
|[Storage, Furnish...|  48|
+--------------------+----+
only showing top 20 rows

+--------------------+-------------+-------------------+------------------+
|          antecedent|   consequent|         confidence|              lift|
+--------------------+-------------+-------------------+------------------+
|[Accessories, Sto...|      [Paper]| 0.4090909090909091|1.7205175177467371

In [11]:
df_output.filter(F.size('prediction') > 0).show(5, False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Order ID          | CA-2015-115798                                                                                                                                                                                                                                                  
 Sub-Category List | [Bookcases, Phones, Paper]                                                                                                                                                                                                                                      
 Product Name List | [BlackBerry Q10, Adams Telephone Message Books, 5 1/4� x 11�, Bestar Classic Bookcase, Innergie mMini Combo Duo USB Travel Charging Kit]         

In [12]:
model \
  .associationRules \
  .groupBy("antecedent") \
  .agg(F.collect_list("consequent").alias("consequent_list")) \
  .withColumn("consequent_list", F.udf(lambda x: [item for sublist in x for item in sublist], ArrayType(StringType()))(F.col("consequent_list"))) \
  .show(5, False)

+---------------------------------+------------------+
|antecedent                       |consequent_list   |
+---------------------------------+------------------+
|[Fasteners, Art]                 |[Paper, Binders]  |
|[Appliances, Chairs, Furnishings]|[Binders]         |
|[Envelopes, Appliances]          |[Binders]         |
|[Appliances, Chairs, Accessories]|[Binders]         |
|[Envelopes, Phones, Paper]       |[Storage, Binders]|
+---------------------------------+------------------+
only showing top 5 rows



In [13]:
# Display frequent itemsets.
model.freqItemsets.filter(F.size('items') > 1).orderBy('freq', ascending=False).show(20, False)

+--------------------------+----+
|items                     |freq|
+--------------------------+----+
|[Paper, Binders]          |275 |
|[Phones, Binders]         |199 |
|[Storage, Binders]        |199 |
|[Furnishings, Binders]    |198 |
|[Storage, Paper]          |178 |
|[Furnishings, Paper]      |177 |
|[Phones, Paper]           |175 |
|[Accessories, Binders]    |161 |
|[Accessories, Paper]      |153 |
|[Art, Paper]              |150 |
|[Art, Binders]            |150 |
|[Phones, Furnishings]     |148 |
|[Storage, Furnishings]    |138 |
|[Chairs, Paper]           |133 |
|[Appliances, Binders]     |130 |
|[Art, Phones]             |124 |
|[Chairs, Binders]         |119 |
|[Storage, Phones]         |117 |
|[Accessories, Phones]     |116 |
|[Accessories, Furnishings]|115 |
+--------------------------+----+
only showing top 20 rows



In [15]:
spark.stop()